# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
df = pd.read_csv("../datasets/final_data/FIFA Data 1990-2018.csv")
df_teams = pd.read_csv("../datasets/final_data/world_cup_comparisons.csv")
fifa_ranking = pd.read_csv("../datasets/final_data/fifa_ranking.csv")
continent = pd.read_csv("../datasets/final_data/countries and continents1.csv",encoding ='utf-8',na_filter = False)





df=df.replace({"South Korea":"Korea Republic"})
df = df.replace({"Iran":"IR Iran"})
df = df.replace({"Soviet Union":"Russia"})
df = df.replace({"Czechoslovakia":"Czech Republic"})


df_teams=df_teams.replace({"South Korea":"Korea Republic"})
df_teams = df_teams.replace({"Iran":"IR Iran"})

fifa_ranking=fifa_ranking.replace({"South Korea":"Korea Republic"})
fifa_ranking = fifa_ranking.replace({"Iran":"IR Iran"})


In [3]:
df.head()

,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,Score T2,Winner
0,Russia,Saudi Arabia,Group Phase,2018,5,0,70.0,457.0,67.0,465.0,1
1,Egypt,Uruguay,Group Phase,2018,0,1,45.0,649.0,14.0,1018.0,2
2,Morocco,IR Iran,Group Phase,2018,0,1,41.0,686.0,37.0,708.0,2
3,Portugal,Spain,Group Phase,2018,3,3,4.0,1274.0,10.0,1126.0,0
4,France,Australia,Group Phase,2018,2,1,7.0,1198.0,36.0,718.0,1


# Add lenth of name for each country

In [ ]:
# len_Team1 =[]
# len_Team2 =[]

# for i in range (len(df['Team1'])):
#     len_Team1.append(len(df['Team1'][i]))
#     len_Team2.append(len(df['Team2'][i]))
    
# df['len_Team1'] =len_Team1
# df['len_Team2'] = len_Team2

# df.head()
    

# Add the Continent

In [4]:
continent = continent.drop_duplicates()
df['Team1'] = df['Team1'].str.strip()
df['Team2'] = df['Team2'].str.strip()

continent['name'] = continent['name'].str.strip()
continent['Continent'] = continent['Continent'].str.strip()

In [5]:
df.loc[df.Year >= 1990,'continent_Team1'] = df['Team1'].map(continent.set_index('name')['Continent'])
df.loc[df.Year >= 1990,'continent_Team2'] = df['Team2'].map(continent.set_index('name')['Continent'])

In [6]:
df.head()

,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,Score T2,Winner,continent_Team1,continent_Team2
0,Russia,Saudi Arabia,Group Phase,2018,5,0,70.0,457.0,67.0,465.0,1,EU,AS
1,Egypt,Uruguay,Group Phase,2018,0,1,45.0,649.0,14.0,1018.0,2,AF,SA
2,Morocco,IR Iran,Group Phase,2018,0,1,41.0,686.0,37.0,708.0,2,AF,AS
3,Portugal,Spain,Group Phase,2018,3,3,4.0,1274.0,10.0,1126.0,0,EU,EU
4,France,Australia,Group Phase,2018,2,1,7.0,1198.0,36.0,718.0,1,EU,OC


In [7]:
# AF = Africa
# AS = Asia
# EU = Europe
# NA = North America
# SA = South America
# OC = Oceania
# AN = Antarctica

continent_dict = {"AF":1, "AS":2, "EU":3, "OC":4, "NA":5, "SA":6, "AN":7}
continent_encoded_team1 = []
continent_encoded_team2 = []


for i in range(len(df.continent_Team1)):
    continent_encoded_team1.append(continent_dict[list(df.continent_Team1)[i]])
    continent_encoded_team2.append(continent_dict[list(df.continent_Team2)[i]])
    
df["continent_en_team1"] = continent_encoded_team1
df["continent_en_team2"] = continent_encoded_team2

df = df.drop(['continent_Team1','continent_Team2'], axis =1)

In [8]:
df.head()

,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,Score T2,Winner,continent_en_team1,continent_en_team2
0,Russia,Saudi Arabia,Group Phase,2018,5,0,70.0,457.0,67.0,465.0,1,3,2
1,Egypt,Uruguay,Group Phase,2018,0,1,45.0,649.0,14.0,1018.0,2,1,6
2,Morocco,IR Iran,Group Phase,2018,0,1,41.0,686.0,37.0,708.0,2,1,2
3,Portugal,Spain,Group Phase,2018,3,3,4.0,1274.0,10.0,1126.0,0,3,3
4,France,Australia,Group Phase,2018,2,1,7.0,1198.0,36.0,718.0,1,3,4


In [9]:
df.isnull().sum()

Team1                 0
Team2                 0
Rounds                0
Year                  0
Score1                0
Score2                0
Ranking T1            0
Score T1              0
Ranking T2            0
Score T2              0
Winner                0
continent_en_team1    0
continent_en_team2    0
dtype: int64

In [11]:
#Adding who is the winner 
winner = []
for i in range (len(df['Team1'])):
    if df['Score1'][i] > df['Score2'][i]:
        winner.append(df['Team1'][i])
    elif df['Score1'][i] < df['Score2'][i]:
        winner.append(df['Team2'][i])
    else:
        winner.append('-1')

df['winner'] = winner
df.head()

,Team1,Team2,Rounds,Year,Score1,Score2,Ranking T1,Score T1,Ranking T2,Score T2,Winner,continent_en_team1,continent_en_team2,winner
0,Russia,Saudi Arabia,Group Phase,2018,5,0,70.0,457.0,67.0,465.0,1,3,2,Russia
1,Egypt,Uruguay,Group Phase,2018,0,1,45.0,649.0,14.0,1018.0,2,1,6,Uruguay
2,Morocco,IR Iran,Group Phase,2018,0,1,41.0,686.0,37.0,708.0,2,1,2,IR Iran
3,Portugal,Spain,Group Phase,2018,3,3,4.0,1274.0,10.0,1126.0,0,3,3,-1
4,France,Australia,Group Phase,2018,2,1,7.0,1198.0,36.0,718.0,1,3,4,France
